In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model

Merge all the data

In [2]:
# Read all the data
building1 = pd.read_csv("../data/schemas/warm_up/Building_1.csv", sep=",")
building2 = pd.read_csv("../data/schemas/warm_up/Building_2.csv", sep=",")
building3 = pd.read_csv("../data/schemas/warm_up/Building_3.csv", sep=",")
weather = pd.read_csv("../data/schemas/warm_up/weather.csv", sep=",")
carbon_intensity = pd.read_csv("../data/schemas/warm_up/carbon_intensity.csv", sep=",")

# Distinguish between the buildings
building1 = building1.add_suffix("_1")
building2 = building2.add_suffix("_2")
building3 = building3.add_suffix("_3")

# Merge
data = [building1, building2, building3, weather, carbon_intensity]
merged = pd.concat(data, axis=1)
merged.head()

,Month_1,Hour_1,Day Type_1,Daylight Savings Status_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,...,6h Outdoor Relative Humidity (%),12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh
0,6,1,5,0,23.098652,-0.123570,61.086187,0.356839,0.055682,1.119216,...,72.980273,41.822361,83.230997,54.625927,116.842886,0.0,143.324335,1020.756093,0.0,0.402488
1,6,2,5,0,22.234742,0.012520,64.361378,0.345078,0.159338,1.469638,...,87.317655,37.910023,95.353948,78.375479,89.220504,0.0,300.361061,825.097292,0.0,0.382625
2,6,3,5,0,22.223061,0.000838,64.493415,0.338769,0.057004,1.458372,...,60.341453,30.190199,66.266409,259.822134,82.855932,0.0,196.691127,987.909330,0.0,0.369458
3,6,4,5,0,22.222251,0.000029,64.769702,0.334856,0.000000,1.337342,...,50.063925,31.519739,94.456370,248.631525,105.130119,0.0,465.483196,854.331825,0.0,0.367017
4,6,5,5,0,22.222236,0.000014,65.152120,0.348607,0.000000,1.163453,...,40.942579,33.119543,100.000000,272.825159,75.627986,0.0,569.203332,941.243325,0.0,0.374040


Drop the columns with constant values

In [3]:
merged_dropped = merged.drop(columns=["Month_1", "Month_2", "Month_3",
                                      "Daylight Savings Status_1", "Daylight Savings Status_2", "Daylight Savings Status_3",
                                      "Heating Load (kWh)_1", "Heating Load (kWh)_2", "Heating Load (kWh)_3",
                                      "HVAC Mode (Off/Cooling/Heating)_1", "HVAC Mode (Off/Cooling/Heating)_2", "HVAC Mode (Off/Cooling/Heating)_3"])

# Insert an index column
index = range(1, len(merged_dropped)+1)
merged_dropped.insert(loc=0, column="Index", value=index)

In [4]:
# Add a column with average solar generation for later prediction
merged_dropped["Avg solar generation"] = merged_dropped[["Solar Generation (W/kW)_1", "Solar Generation (W/kW)_2", "Solar Generation (W/kW)_3"]].mean(axis=1)

Split into train-test and normalize

In [5]:
train, test = train_test_split(merged_dropped, test_size=0.05, shuffle=False)
columns = list(merged_dropped)
columns.remove("Index")

for column in columns:
    min_of_column = train[column].min()
    max_of_column = train[column].max()
    train[column] = (train[column] - min_of_column) / (max_of_column - min_of_column)
    test[column] = (test[column] - min_of_column) / (max_of_column - min_of_column)

Export the train and test sets

In [6]:
train.to_csv("../data/schemas/warm_up/TrainSet.csv", index=False)
test.to_csv("../data/schemas/warm_up/TestSet.csv", index=False)

In [7]:
test.head()

,Index,Hour_1,Day Type_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,Solar Generation (W/kW)_1,...,12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh,Avg solar generation
684,685,0.521739,0.666667,0.315786,0.501662,0.736918,0.011680,0.0,0.610427,0.942854,...,0.728211,0.200902,0.132386,0.0,0.400481,0.240234,0.0,0.558196,0.471615,0.942854
685,686,0.565217,0.666667,0.613583,0.381025,0.392378,0.011713,0.0,0.445242,0.936498,...,0.963312,0.045915,0.027224,0.0,0.548163,0.028463,0.0,0.506819,0.518917,0.936498
686,687,0.608696,0.666667,0.615364,0.402745,0.393064,0.005136,0.0,0.472410,0.849920,...,0.828332,0.129598,0.000000,0.0,0.945442,0.000000,0.0,0.481153,0.554976,0.849920
687,688,0.652174,0.666667,0.615386,0.403023,0.389506,0.011964,0.0,0.567253,0.788518,...,0.955521,0.245467,0.000000,0.0,0.757120,0.000000,0.0,0.396474,0.564855,0.788518
688,689,0.695652,0.666667,0.615389,0.403053,0.390591,0.156114,0.0,0.643627,0.594962,...,0.625298,0.136356,0.000000,0.0,0.749071,0.000000,0.0,0.294987,0.555823,0.594962
